In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

## Pembacaan dataset

In [13]:
file_path = 'preeclampsia.csv'
dataset = pd.read_csv(file_path)

print(dataset.head())

   age  gest_age  height  weight        bmi  sysbp  diabp         hb  \
0   28        11     152      97  28.508125     71    133  14.683883   
1   28        12     157      70  27.606337     87    127  14.510683   
2   21        11     173      98  25.673333     76    128  12.592319   
3   30        12     148      65  27.367781     82    122  14.128237   
4   23        11     157      71  21.585036     76    139  13.089027   

         pcv       tsh  ...       pp_13  glycerides  htn  diabetes  fam_htn  \
0  39.232347  2.764080  ...   61.197945  257.668101    0         0        1   
1  38.215125  2.439556  ...   95.611529  269.812342    1         0        1   
2  39.245587  4.147177  ...   26.671472  197.074922    1         1        0   
3  37.276060  2.118313  ...   83.365411  244.343060    0         0        0   
4  40.280974  1.186492  ...  117.633252  239.124513    1         1        1   

   sp_art  occupation  diet  activity  sleep  
0       0           1     0         2      1 

## Pembuatan label baru -> Risk Level & Risk_Level_Num
Hal ini perlu dilakukan untuk mengklasifikasikan setiap baris data termasuk ke dalam low risk atau high risk sehingga model dapat mempelajari data dengan menjadikan label baru sebagai target klasifikasi nantinya. Preprocessing data tidak diperlukan karena dataset sudah layak untuk digunakan dan fitur-fitur yang berpengaruh adalah fitur yang memiliki value numerik.

In [16]:
def classify_risk(row):
    if (row['sysbp'] > 90 or 
        row['diabp'] > 140 or 
        row['bmi'] >= 32 or  
        row['hb'] < 9.5 or  
        (row['fam_htn'] == 1 and row['sysbp'] > 85) or  
        row['diabetes'] == 1):
        return 'High Risk'
    else:
        return 'Low Risk'

dataset['Risk_Level'] = dataset.apply(classify_risk, axis=1)

In [18]:
dataset['Risk_Level_Num'] = dataset['Risk_Level'].apply(lambda x: 1 if x == 'High Risk' else 0)

In [20]:
dataset

,age,gest_age,height,weight,bmi,sysbp,diabp,hb,pcv,tsh,...,htn,diabetes,fam_htn,sp_art,occupation,diet,activity,sleep,Risk_Level,Risk_Level_Num
0,28,11,152,97,28.508125,71,133,14.683883,39.232347,2.764080,...,0,0,1,0,1,0,2,1,Low Risk,0
1,28,12,157,70,27.606337,87,127,14.510683,38.215125,2.439556,...,1,0,1,1,2,1,2,2,High Risk,1
2,21,11,173,98,25.673333,76,128,12.592319,39.245587,4.147177,...,1,1,0,1,2,0,1,1,High Risk,1
3,30,12,148,65,27.367781,82,122,14.128237,37.276060,2.118313,...,0,0,0,0,0,0,0,0,Low Risk,0
4,23,11,157,71,21.585036,76,139,13.089027,40.280974,1.186492,...,1,1,1,0,1,1,2,2,High Risk,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,24,12,146,100,22.661248,87,130,11.136807,39.361406,3.864517,...,0,0,0,1,2,1,0,1,Low Risk,0
396,18,11,144,45,32.477823,81,124,10.425359,38.723635,4.968701,...,0,1,1,1,2,0,2,1,High Risk,1
397,25,11,145,107,18.335092,89,136,15.236094,37.205125,2.978449,...,1,0,1,0,1,1,1,2,High Risk,1
398,24,12,157,47,34.690710,72,120,15.618601,43.821312,4.113711,...,0,0,0,0,2,1,1,2,High Risk,1


Kolom/fitur yang dimasukkan ke dalam algoritma pelatihan model hanya kolom/fitur numerik

In [23]:
X = dataset.drop(columns=['Risk_Level', 'Risk_Level_Num'])
X = X.select_dtypes(include=['float64', 'int64']) 
y = dataset['Risk_Level_Num']

In [25]:
X

,age,gest_age,height,weight,bmi,sysbp,diabp,hb,pcv,tsh,...,pp_13,glycerides,htn,diabetes,fam_htn,sp_art,occupation,diet,activity,sleep
0,28,11,152,97,28.508125,71,133,14.683883,39.232347,2.764080,...,61.197945,257.668101,0,0,1,0,1,0,2,1
1,28,12,157,70,27.606337,87,127,14.510683,38.215125,2.439556,...,95.611529,269.812342,1,0,1,1,2,1,2,2
2,21,11,173,98,25.673333,76,128,12.592319,39.245587,4.147177,...,26.671472,197.074922,1,1,0,1,2,0,1,1
3,30,12,148,65,27.367781,82,122,14.128237,37.276060,2.118313,...,83.365411,244.343060,0,0,0,0,0,0,0,0
4,23,11,157,71,21.585036,76,139,13.089027,40.280974,1.186492,...,117.633252,239.124513,1,1,1,0,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,24,12,146,100,22.661248,87,130,11.136807,39.361406,3.864517,...,36.487012,177.844590,0,0,0,1,2,1,0,1
396,18,11,144,45,32.477823,81,124,10.425359,38.723635,4.968701,...,126.482298,230.786530,0,1,1,1,2,0,2,1
397,25,11,145,107,18.335092,89,136,15.236094,37.205125,2.978449,...,89.894014,195.257282,1,0,1,0,1,1,1,2
398,24,12,157,47,34.690710,72,120,15.618601,43.821312,4.113711,...,94.907792,225.430970,0,0,0,0,2,1,1,2


## Pelatihan model
Algoritma yang dilakukan dalam pelatihan model adalah Random Forest Classifier. Namun sebelum ditrain, akan dilakukan hyperparameter tuning terlebih dahulu untuk mencari hyperparameter terbaik yang dapat membuat kinerja model optimal.

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
rf_model = RandomForestClassifier(random_state=42)
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=10, cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   random_state=42)

In [31]:
best_rf_model = random_search.best_estimator_
best_rf_model.fit(X_train, y_train)

y_pred = best_rf_model.predict(X_test)
y_pred_proba = best_rf_model.predict_proba(X_test)[:, 1]

In [32]:
best_rf_model

RandomForestClassifier(bootstrap=False, max_depth=30, min_samples_leaf=2,
                       min_samples_split=10, random_state=42)

#### Hyperparameter terbaik
- bootstrap = False 
- max_depth = 30 
- min_samples_leaf = 2
- min_samples_split = 10
- random_state = 42

## Laporan Klasifikasi
Evaluasi kinerja model menggunakan score precision, recall, F1, ROC AUC, accuracy, dan confusion matrix.

In [38]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix

y_pred = best_rf_model.predict(X_test)
y_pred_proba = best_rf_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
report = classification_report(y_test, y_pred, target_names=['Low Risk', 'High Risk'])
conf_matrix = confusion_matrix(y_test, y_pred)

# Menampilkan hasil evaluasi
print("Classification Report:")
print(report)

print(f"Accuracy: {accuracy:.2f}")
print(f"ROC AUC Score: {roc_auc:.2f}")
print("Confusion Matrix:")
print(conf_matrix)

Classification Report:
              precision    recall  f1-score   support

    Low Risk       0.93      1.00      0.96        25
   High Risk       1.00      0.96      0.98        55

    accuracy                           0.97        80
   macro avg       0.96      0.98      0.97        80
weighted avg       0.98      0.97      0.98        80

Accuracy: 0.97
ROC AUC Score: 1.00
Confusion Matrix:
[[25  0]
 [ 2 53]]


### Precision Score
Low risk precision = 0.93 -> Menunjukkan bahwa dari semua prediksi yang dilakukan, 93% klasifikasi low risk adalah benar

High risk precision = 1.00 -> Menunjukkan bahwa dari semua prediksi yang dilakukan, 100% klasifikasi high risk adalah benar

Kesimpulan: Angka yang tinggi pada precision membuktikan bahwa model berhasil melakukan minimalisasi jumlah False Positive. Terutama, model berhasil mencapai 100% akurasi dalam mengidentifikasi resiko tinggi.

### Recall score
Low Risk Recall = 1.00 -> Dari semua kasus yang sebenarnya Low Risk, 100% teridentifikasi dengan benar sebagai Low Risk.

High Risk Recall = 0.96 -> Dari semua kasus yang sebenarnya High Risk, 96% teridentifikasi dengan benar sebagai High Risk.

Kesimpulan: Recall tinggi menunjukkan bahwa model sangat baik dalam mendeteksi semua kasus True Positives. Dalam konteks ini, recall yang tinggi untuk High Risk menunjukkan bahwa model sangat efektif dalam mendeteksi hampir semua kasus risiko tinggi.

### F1 score
Low Risk F1-Score = 0.96 -> Kombinasi harmonis dari precision dan recall untuk kelas Low Risk.

High Risk F1-Score = 0.98 -> Kombinasi harmonis dari precision dan recall untuk kelas High Risk.

Kesimpulan: F1-Score memberikan keseimbangan antara precision dan recall. Nilai yang tinggi menunjukkan bahwa model memiliki keseimbangan yang baik antara mendeteksi kasus risiko tinggi dan menghindari prediksi yang salah.

### Accuracy
Akurasi dari model ini adalah 0.97, yang berarti 97% dari total prediksi yang dilakukan oleh model adalah benar.

Kesimpulan: Akurasi yang tinggi menunjukkan bahwa model secara keseluruhan bekerja dengan baik dalam mengklasifikasikan Low Risk dan High Risk.

### ROC AUC Score
ROC AUC (1.00) menunjukkan kemampuan model dalam membedakan antara kelas Low Risk dan High Risk.
Nilai AUC yang sempurna (1.00) berarti model ini dapat membedakan antara kedua kelas dengan sangat baik.

### Confusion Matrix
[[25, 0], [2, 53]] menunjukkan bahwa:

25 prediksi benar untuk Low Risk (True Negatives).

53 prediksi benar untuk High Risk (True Positives).

2 prediksi salah untuk High Risk sebagai Low Risk (False Negatives).

0 prediksi salah untuk Low Risk sebagai High Risk (False Positives).

Confusion matrix menunjukkan bahwa model melakukan pekerjaan yang sangat baik dalam memisahkan kedua kelas, dengan hanya 2 kasus salah dari total 80.

In [46]:
import joblib
joblib.dump(best_rf_model, 'model.pkl')

['model.pkl']